In [2]:
#updaing products table hw 5 

#connecting to database and creating ConformedProducts.txt table
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('products.db')
cursor = conn.cursor()

# Create a temporary table to store the data from ConformedProducts.txt
cursor.execute("""
    CREATE TEMP TABLE temp_products (
        ProductKey1 INTEGER,
        sku TEXT,
        product_name TEXT,
        product_class_id INTEGER,
        subcategory TEXT,
        category TEXT,
        department TEXT,
        product_family TEXT,
        size TEXT,
        brandName TEXT,
        supplier TEXT
    )
""")

# Load data from ConformedProducts.txt into the temporary table
with open('ConformedProducts.txt', 'r') as file:
    next(file)  # Skip header line
    for line in file:
        fields = line.strip().split('\t')
        cursor.execute("""
            INSERT INTO temp_products (ProductKey1, sku, product_name, product_class_id, subcategory, category, department, product_family, size, brandName, supplier)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, fields)


In [ ]:
# Update records with null values in the subcategory column (DO NOT RUN THIS YET)
cursor.execute("""
    UPDATE temp_products
    SET subcategory = 'YourDefaultValue'
    WHERE subcategory IS NULL
""")



In [ ]:
#this is to update the ConformedProducts.txt file
#MAY OR MAY NOT NEED

# Update subcategory based on product name
cursor.execute("""
    UPDATE products
    SET subcategory = (
        SELECT subcategory
        FROM temp_product_class
        WHERE products.product_name = temp_product_class.product_name
    )
    WHERE EXISTS (
        SELECT 1
        FROM temp_product_class
        WHERE products.product_name = temp_product_class.product_name
    )
""")

# Handle null subcategory values
cursor.execute("""
    UPDATE products
    SET subcategory = 'Other'
    WHERE subcategory IS NULL
""")

# Update subcategory for obsolete item types
obsolete_item_types = ['Obsolete_Type1', 'Obsolete_Type2', 'Obsolete_Type3']  # Example list of obsolete item types
for item_type in obsolete_item_types:
    cursor.execute("""
        UPDATE products
        SET subcategory = 'Other'
        WHERE item_type = ?
    """, (item_type,))

# Commit the changes and close the connection
conn.commit()
conn.close()